In [17]:
import chromadb
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import StorageContext
from llama_index.core import VectorStoreIndex

chroma_client = chromadb.EphemeralClient()
chroma_collection = chroma_client.create_collection("quickstart")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

In [4]:
v = VectorStoreIndex(
    index_struct=VectorStoreIndex.index_struct_cls(index_id='test'),
    storage_context=storage_context,
    embed_model='skip'
)
retriever = v.as_retriever()

In [16]:

retriever._vector_store.client._client.delete_collection(retriever._vector_store.client.name)

In [19]:
retriever._vector_store._collection = chroma_collection

In [22]:
from llama_index.core.schema import MetadataMode, TextNode
retriever._vector_store.add([TextNode(text="test", embedding=[1,0,0,0])])

['1ac4550a-a88c-4226-82e3-4804cab047d0']

In [ ]:
import os
import chromadb
from llama_index.vector_stores.chroma import ChromaVectorStore

for i in range(1,5):
    db_path = f'{i}_chroma'
    chroma_client = chromadb.PersistentClient(path=db_path)
    chroma_collection = chroma_client.get_or_create_collection(name='quickstart')
    vector_stores.append(ChromaVectorStore(chroma_collection=chroma_collection))